In [56]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import pandas_datareader.data as web
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import pandas_ta as pta
from scipy import stats

In [57]:
# plt.figure(figsize = (10,5))
# plt.plot(df.High)

In [58]:
ticks_df = pd.read_csv(".//input_files//ticker_list.csv")
#filter to only small and mid_cap stocks
ticks_df = ticks_df[ticks_df.cap_size != 'large']
print("stocks after filter = ", ticks_df.shape[0])

stocks after filter =  5992


In [59]:
run_state = False

start = (date.today()-timedelta(365))
end = date.today()

ticks_list = ticks_df.ticker.to_list()
ticks_list.append('^GSPC')

if run_state:
    stk_df = web.DataReader(ticks_list, 'yahoo', start, end)
    stk_df.to_csv(".//input_files//stockprice_data.csv")

    # snp500 = web.DataReader(['^GSPC'], 'yahoo', start, end)
    # snp500.to_csv(".//input_files//snp500.csv")
else:
    stk_df = pd.read_csv(".//input_files//stockprice_data.csv",header=[0,1], index_col=0)
    
stk_df.index = pd.to_datetime(stk_df.index)

stk_df = stk_df[['Adj Close']] #taking only the tail 64 since 64 trading days ~= 90 days
stk_df

Attributes Adj Close                                                      \
Symbols         SNGX   CORZ       OTEX       RADI        EXLS        OLO   
Date                                                                       
2021-10-15      1.10  10.68  48.896538  16.850000  124.150002  27.940001   
2021-10-18      1.06  10.68  48.984478  17.100000  125.830002  27.410000   
2021-10-19      1.06  10.60  49.082195  17.360001  124.129997  26.770000   
2021-10-20      1.05  10.98  49.258080  17.320000  123.699997  26.230000   
2021-10-21      1.03  11.00  49.316711  17.430000  122.839996  27.040001   
...              ...    ...        ...        ...         ...        ...   
2022-10-10      0.58   1.29  26.190001   8.700000  155.309998   7.930000   
2022-10-11      0.59   1.27  25.500000   8.770000  156.610001   7.750000   
2022-10-12      0.57   1.24  25.740000   8.560000  153.979996   8.100000   
2022-10-13      0.55   1.24  26.020000   8.740000  156.910004   7.990000   
2022-10-14      0.59   1.11  25.760000   8.190000  152.229996   7.640000   

Attributes                                          ...                        \
Symbols           EEFT       SPWR       OFED  LNDC  ...         KW       TARO   
Date                                                ...                         
2021-10-15  135.440002  27.540001  24.365740  9.81  ...  22.412367  63.830002   
2021-10-18  135.449997  27.440001  24.365740  9.58  ...  22.352970  62.279999   
2021-10-19  134.839996  29.570000  24.365740  9.54  ...  22.253975  62.439999   
2021-10-20  132.009995  28.799999  24.729261  9.51  ...  22.352970  62.410000   
2021-10-21  123.489998  29.760000  24.729261  9.68  ...  22.323271  63.889999   
...                ...        ...        ...   ...  ...        ...        ...   
2022-10-10   75.889999  21.469999  24.049999  8.05  ...  14.490000  28.400000   
2022-10-11   74.010002  20.309999  24.150000  8.23  ...  14.710000  28.459999   
2022-10-12   73.769997  18.600000  24.150000  8.06  ...  14.470000  29.000000   
2022-10-13   75.739998  17.959999  24.150000  8.24  ...  14.920000  30.500000   
2022-10-14   75.820000  16.730000  24.150000  8.18  ...  14.900000  30.340000   

Attributes                                                               
Symbols     GURE  MRAM  MVST        BST        KBR        ^GSPC EAI EMP  
Date                                                                     
2021-10-15  5.76  6.02  7.61  50.917503  42.316467  4471.370117 NaN NaN  
2021-10-18  5.95  6.20  8.23  51.261093  42.476078  4486.459961 NaN NaN  
2021-10-19  5.95  6.13  8.10  51.738297  42.715492  4519.629883 NaN NaN  
2021-10-20  5.96  6.07  8.10  51.738297  42.725468  4536.189941 NaN NaN  
2021-10-21  5.55  6.05  8.09  51.776474  43.224251  4549.779785 NaN NaN  
...          ...   ...   ...        ...        ...          ...  ..  ..  
2022-10-10  3.30  5.53  1.81  29.440001  45.820000  3612.389893 NaN NaN  
2022-10-11  3.28  5.35  1.80  28.780001  46.110001  3588.840088 NaN NaN  
2022-10-12  3.23  5.15  1.74  28.870001  44.939999  3577.030029 NaN NaN  
2022-10-13  3.19  5.20  1.68  29.059999  46.369999  3669.909912 NaN NaN  
2022-10-14  3.18  5.05  1.57  28.350000  44.970001  3583.070068 NaN NaN  

[252 rows x 5993 columns]

#### Condition 1 

In [60]:
snp_30daygain = ((stk_df.loc[stk_df.index[-1],('Adj Close','^GSPC')] - stk_df.loc[stk_df.index[-22],('Adj Close','^GSPC')])/stk_df.loc[stk_df.index[-22],('Adj Close','^GSPC')])

#condition1a = stocks that made 30% increase in 3 months
condition1a = ((stk_df.iloc[[-1],:] - stk_df.iloc[-22*3,:])/stk_df.iloc[-22*3,:]) >= 0.3 

#condition1b = stocks that gained more than snp500 in 30 days
condition1b = ((stk_df.iloc[[-1],:] - stk_df.iloc[-22,:])/stk_df.iloc[-22,:]) >= snp_30daygain #22 working days in a 30 calendar days

In [61]:
bool_mask = (condition1a + condition1b).values[0]
stk_df = stk_df.iloc[:,bool_mask] #stocks filtered to condition 1a&b

#### Condition 2

In [62]:
rsi65_df = stk_df[['Adj Close']].apply(lambda x: pta.rsi(x,length = 65))
rs65_df = rsi65_df.div(rsi65_df[('Adj Close','^GSPC')] , axis = 0)
#condition 2a = stocks that have a RS > 1.3 over 65 day period (3 calendar months)
condition2a = rs65_df.iloc[-1,:] >= 1.3

rsi260_df = stk_df[['Adj Close']].apply(lambda x: pta.rsi(x,length = 252))
rs260_df = rsi260_df.div(rsi260_df[('Adj Close','^GSPC')] , axis = 0)
# condition 2b = stocks that have a RS > 1.3 over 260 day period (1 calendar year)
condition2b = rs260_df.iloc[-1,:] >= 1.3

In [63]:
bool_mask = (condition2a + condition2b).values
stk_df = stk_df.iloc[:,bool_mask] #stocks filtered to condition 1a&b and 2a&b


#### Condition 3

In [64]:
ewm21_fd = stk_df.ewm(span=21, adjust=False).mean()

# condition 3 = 21 EMW > recent stock prices
condition3 = (stk_df.tail() >= ewm21_fd.tail()).all(axis='index') 

In [65]:
bool_mask = (condition3).values
stk_df = stk_df.iloc[:,bool_mask] 
ewm21_fd = ewm21_fd.iloc[:,bool_mask] 

In [66]:
stk_df.tail()

Attributes Adj Close                                                     \
Symbols         GDNR   NGC   VPCB  KAIR        BSM   DHAC   SHLX   ATCO   
Date                                                                      
2022-10-10     10.00  9.82  9.920  9.98  17.190001  10.17  15.82  14.70   
2022-10-11     10.01  9.83  9.940  9.98  17.160000  10.18  15.82  14.71   
2022-10-12     10.01  9.84  9.920  9.98  17.100000  10.17  15.82  14.77   
2022-10-13     10.01  9.85  9.935  9.98  17.500000  10.17  15.82  14.78   
2022-10-14     10.01  9.85  9.935  9.98  17.160000  10.18  15.82  14.85   

Attributes                         ...                                         \
Symbols          PLRX         FCN  ...   NERV   VCKA  RKTA   PLAO   POW  WPCA   
Date                               ...                                          
2022-10-10  22.000000  174.380005  ...  10.47  10.35  9.94  10.20  9.97  9.93   
2022-10-11  23.020000  178.639999  ...  12.46  10.35  9.93  10.19  9.97  9.95   
2022-10-12  22.629999  173.259995  ...  13.19  10.35  9.92  10.19  9.97  9.95   
2022-10-13  23.309999  179.320007  ...  12.48  10.39  9.92  10.19  9.97  9.95   
2022-10-14  22.580000  179.960007  ...  13.43  10.39  9.92  10.19  9.97  9.95   

Attributes                             
Symbols     MITA   CPAQ   AHPA   PTIC  
Date                                   
2022-10-10  9.85  10.11  10.26  9.980  
2022-10-11  9.86  10.09  10.27  9.975  
2022-10-12  9.86  10.10  10.28  9.970  
2022-10-13  9.86  10.10  10.28  9.970  
2022-10-14  9.86  10.10  10.28  9.970  

[5 rows x 454 columns]

#### Condition 4

In [70]:
#condition 4a = check if slope of 21 ewm > 0.15
slope21_df = stk_df[['Adj Close']].tail(21).apply(lambda x: stats.linregress(range(len(x)), x)[0])
condition4a = slope21_df >= 0.15

#condition 4b = check if slope of 14 ewm > 0.15
slope14_df = stk_df[['Adj Close']].tail(14).apply(lambda x: stats.linregress(range(len(x)), x)[0])
condition4b = slope14_df >= 0.15

#condition 4c = check if slope of 7 ewm > 0.15
slope7_df = stk_df[['Adj Close']].tail(7).apply(lambda x: stats.linregress(range(len(x)), x)[0])
condition4c = slope7_df >= 0.15

In [73]:
bool_mask = (condition4a*condition4b*condition4c).values
stk_df = stk_df.iloc[:,bool_mask] 

In [74]:
stk_df

Attributes   Adj Close                                                         \
Symbols            FCN       SIGI       URGN       INBX       HURN        WWE   
Date                                                                            
2021-10-15  143.669998  77.463509  17.020000  39.709999  51.299999  59.341648   
2021-10-18  145.009995  76.339706  16.459999  39.820000  50.450001  59.551125   
2021-10-19  144.850006  75.994682  17.170000  39.910000  49.560001  59.820450   
2021-10-20  144.190002  76.872040  16.559999  38.459999  49.169998  60.119701   
2021-10-21  142.330002  77.404358  16.670000  36.560001  49.810001  60.089779   
...                ...        ...        ...        ...        ...        ...   
2022-10-10  174.380005  86.309998  10.650000  29.920000  69.570000  73.949997   
2022-10-11  178.639999  86.849998  10.770000  30.910000  69.989998  74.940002   
2022-10-12  173.259995  86.160004  11.440000  31.100000  68.860001  74.550003   
2022-10-13  179.320007  88.800003  11.310000  33.110001  71.529999  75.489998   
2022-10-14  179.960007  89.459999  11.500000  29.850000  69.440002  75.709999   

Attributes                                      ...                         \
Symbols      SLNG   VIGL        PBF       DICE  ...       ADEA        KNSL   
Date                                            ...                          
2021-10-15   7.00    NaN  15.680000  26.700001  ...   4.925926  165.380005   
2021-10-18   6.98    NaN  15.400000  29.530001  ...   4.825397  165.669998   
2021-10-19   6.94    NaN  15.670000  27.049999  ...   4.804233  165.350006   
2021-10-20   6.81    NaN  15.630000  31.410000  ...   4.846561  162.820007   
2021-10-21   6.43    NaN  15.380000  31.980000  ...   4.804233  165.600006   
...           ...    ...        ...        ...  ...        ...         ...   
2022-10-10   9.72   9.35  38.240002  24.650000  ...   9.690000  271.950012   
2022-10-11  10.92  10.16  37.869999  40.000000  ...   9.010000  274.559998   
2022-10-12  10.85  13.92  40.610001  39.009998  ...   9.610000  270.029999   
2022-10-13  10.06  15.26  42.490002  42.230000  ...  10.670000  282.600006   
2022-10-14   9.87  15.97  40.060001  37.509998  ...  10.420000  275.730011   

Attributes                                                                    \
Symbols          CCRN   FGEN        ADUS         RGA   IMUX       CHCO  ILAG   
Date                                                                           
2021-10-15  18.889999  11.04   77.000000  119.351021   8.67  76.519997   NaN   
2021-10-18  19.709999  10.69   77.000000  118.393524   8.47  77.360001   NaN   
2021-10-19  19.750000  10.75   77.199997  118.146751   8.49  77.360001   NaN   
2021-10-20  19.940001  10.31   78.389999  118.817978   8.43  78.190002   NaN   
2021-10-21  20.150000  10.60   79.500000  117.791389   8.36  80.430000   NaN   
...               ...    ...         ...         ...    ...        ...   ...   
2022-10-10  32.259998  13.96   98.500000  131.789993   6.20  90.489998  4.81   
2022-10-11  32.349998  14.26   98.870003  130.899994   9.41  91.029999  5.32   
2022-10-12  32.209999  14.14  101.029999  130.119995   9.67  90.580002  4.96   
2022-10-13  33.250000  15.42  104.769997  136.500000  10.80  93.059998  5.26   
2022-10-14  32.070000  15.87  102.250000  135.660004  10.03  93.269997  6.51   

Attributes         
Symbols      NERV  
Date               
2021-10-15  10.80  
2021-10-18  10.36  
2021-10-19  10.32  
2021-10-20  10.48  
2021-10-21  10.40  
...           ...  
2022-10-10  10.47  
2022-10-11  12.46  
2022-10-12  13.19  
2022-10-13  12.48  
2022-10-14  13.43  

[252 rows x 21 columns]